## Amplitude Modulation
In the previous parts of the question, you have been able to show the effects of modulating a simple base signal with a carrier signal of higher frequency. Now we will show them visually to appreciate it fully.

### a) Base and carrier signals

In [ ]:
#Run these to import all of the necessary files
import numpy as np
import cmath
import matplotlib as plt
from pylab import *
# Plots graphs in the  
%matplotlib inline

In [ ]:
#Create a 3 second .wav file of the Major Triad
N = 500; #sampling frequency: 1000 Hz, i.e., 1000 samples per second
n = np.arange(0, 2*N+1, 1, dtype = np.float).transpose()
base_freq = 2; #base frequency: 10 Hz
carr_freq = 25; #carrier frequency: 50Hz
x_base = np.cos(2*pi*base_freq/N*n) #base signal
x_carr = np.cos(2*pi*carr_freq/N*n) #carrier signal
# Time domain
figure(figsize=(8,1.5)),plt.plot(n/N, x_base);
plt.title('Base signal: time domain');plt.xlabel('Time (seconds)');plt.ylabel('Signal strength');plt.grid(True)
figure(figsize=(8,1.5)),plt.plot(n/N, x_carr);
plt.title('Carrier signal: time domain');plt.xlabel('Time (seconds)');plt.ylabel('Signal strength');plt.grid(True)
plt.show()

Now, we implement the DFT transformation the DFT coefficients of the above signals:

In [ ]:
def DFT_matrix(num_samples):
    """
    We assume num_samples is odd
    """
    assert np.mod(num_samples, 2) == 1, "num_samples is odd"
    
    #Init Matrix
    U = np.zeros((num_samples,num_samples))
    dft_indices = np.arange(num_samples)-(num_samples-1)/2
    w = cmath.rect(1, 2*pi/num_samples) #Return the complex number x with polar coordinates r and phi. Equivalent to r * (math.cos(phi) + math.sin(phi)*1j)
    #Obtain the DFT matrix
    for k in np.arange(num_samples):
        u_k = np.zeros(num_samples)
        for m in np.arange(num_samples):
            u_k[m] = np.power(w, m*k) 
        U[:,k] = u_k
    
    U_conj = np.conjugate(U)

    return (U,U_conj)

In [ ]:
# Obtain the base and carrier signals in frequency domain
num_samples = len(x_base)
U,U_conj = DFT_matrix(num_samples)
x_base_freq = U_conj.dot(x_base)
x_carr_freq = U_conj.dot(x_carr)
dft_indices = np.concatenate(((np.arange((num_samples-1)/2), np.arange((num_samples+1)/2)-(num_samples-1)/2)),axis=0)
# Frequency domain
figure(figsize=(8,1.5)),plt.stem(dft_indices, np.real(x_base_freq), use_line_collection=True);
plt.title('Base signal: frequency domain');plt.xlabel('DFT component indices');plt.ylabel('Signal strength');plt.grid(True)
figure(figsize=(8,1.5)),plt.stem(dft_indices, np.real(x_carr_freq), use_line_collection=True);
plt.title('Carrier signal: frequency domain');plt.xlabel('DFT component indices');plt.ylabel('Signal strength');plt.grid(True)
plt.show()


Compare the above result with your calculations. You can ignore the small ripples, which results from the finite sample effect (they will eventually disappear as we have more periods of the data). 

### b) Amplitude Modulation

Now we will mix the carrier and base signals, and see what will be the output in time and frequency domains.

In [ ]:
# Your Code Here #
### Solution ###
mix_signal =  # yours to implement: the signal after mixing
### Solution End ###
# End Code #
# time domain plot
figure(figsize=(8,1.5)),plt.plot(n/N, mix_signal),plt.plot(n/N, x_base,'g--'),plt.plot(n/N, -x_base,'g--');
plt.title('Mixed signal: time domain');plt.xlabel('time(seconds)');plt.ylabel('Signal strength');plt.grid(True)
# frequency domain
mix_signal_freq = U_conj.dot(mix_signal)
figure(figsize=(8,1.5)),plt.stem(dft_indices, mix_signal_freq);
plt.title('Mixed signal: frequency domain');plt.xlabel('DFT component indices');plt.ylabel('Signal strength');plt.grid(True)


### c) Demodulation

Now that the modulated signal is successfully received, we need to attract the original base signal out. In the question, you are suggested to mix it with the carrier signal first, let's see what do we get:

In [ ]:
# Your Code Here #
### Solution ###
demodulated_signal = # yours to implement: the received signal mixed with the carrier
### Solution End ###
# End Code #
# time domain plot
figure(figsize=(8,1.5)),plt.plot(n/N, demodulated_signal),plt.plot(n/N, x_base,'g--');
plt.title('Demodulated signal: time domain');plt.xlabel('time(seconds)');plt.ylabel('Signal strength');plt.grid(True)
# frequency domain
demod_signal_freq = U_conj.dot(demodulated_signal)
figure(figsize=(8,1.5)),plt.stem(dft_indices, demod_signal_freq);
plt.title('Demodulated signal: frequency domain');plt.xlabel('DFT component indices');plt.ylabel('Signal strength');plt.grid(True)


Now if you compare the above plots in the time and frequency domains with those for the base signal, what have you noticed? In the above example, you can realize you can exact the signal by only keeping those with frequencies less than a threshold, also known as the cut-off frequency, in other words, you want to multiply each DFT basis coefficient with 0 if it is high frequency and 1 otherwise. (This is called a low-pass filter)

In [ ]:
def low_pass_filter_circulant_matrix(U,U_conj,cutoff_freq):
    num_samples = U.shape[1]
    Lambda = np.zeros(num_samples)
    Lambda[np.int_(np.arange(np.floor(cutoff_freq)))] = 1
    Lambda[np.int_(num_samples-np.arange(np.floor(cutoff_freq))-1)] = 1
    Lambda = np.diag(Lambda)
    
    C_lpf = np.mat(U)*np.mat(Lambda)*np.mat(U_conj)
    return C_lpf,Lambda

Now let's take a look what does the low-pass filter look like for the cirulant matrix and DFT coefficient matrix:

In [ ]:
cutoff_freq = 10
C_lpf,Lambda = low_pass_filter_circulant_matrix(U,U_conj,cutoff_freq)
# time domain plot
figure(figsize=(8,1.5)),plt.plot(dft_indices/N, C_lpf[:,0]);
plt.title('Low pass filter in time domain');plt.xlabel('time(seconds)');plt.ylabel('Value');plt.grid(True)
# frequency domain
figure(figsize=(8,1.5)),plt.plot(dft_indices, np.diag(Lambda) );
plt.title('Low pass filter in frequency domain');plt.xlabel('DFT components');plt.ylabel('Value');plt.grid(True)


In [ ]:
x_demod_lpf = C_lpf.dot(demodulated_signal).reshape((C_lpf.shape[1], 1))/(pow(num_samples,0.5)*16)
plotind = n/N
# time domain plot
figure(figsize=(8,1.5)),plt.plot(plotind.reshape((C_lpf.shape[1],1)), x_demod_lpf);
plt.title('Extracted signal: time domain');plt.xlabel('time(seconds)');plt.ylabel('Signal strength');plt.grid(True)
figure(figsize=(8,1.5)),plt.plot(n/N, x_base,'g--');
plt.title('Original signal: time domain');plt.xlabel('time(seconds)');plt.ylabel('Signal strength');plt.grid(True)
